In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def add_columns(sheet_name, credentials_path, num_columns):
    # Authenticate with Google Sheets
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
    client = gspread.authorize(creds)

    # Open the sheet
    sheet = client.open(sheet_name).sheet1

    # Add columns
    sheet.add_cols(num_columns)

# Usage
sheet_name = 'pixel_art'
credentials_path = 'google.json'
num_columns = 1000  # Number of columns to add

add_columns(sheet_name, credentials_path, num_columns)

In [21]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from PIL import Image
import numpy as np

def resize_image(image_path, new_width, new_height):
    img = Image.open(image_path)
    img = img.resize((new_width, new_height), Image.NEAREST)
    return img

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

def update_sheet_with_pixel_art(image_path, sheet_name, credentials_path, new_width, new_height):
    # Resize image
    img = resize_image(image_path, new_width, new_height)
    
    # Convert image to numpy array
    img_array = np.array(img)

    # Authenticate with Google Sheets
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
    client = gspread.authorize(creds)

    # Open the sheet
    sheet = client.open(sheet_name).sheet1

    # Prepare batch update request
    requests = []

    # Clear existing content
    requests.append({
        'updateCells': {
            'range': {
                'sheetId': sheet.id,
                'startRowIndex': 0,
                'endRowIndex': new_height,
                'startColumnIndex': 0,
                'endColumnIndex': new_width
            },
            'fields': 'userEnteredValue,userEnteredFormat.backgroundColor'
        }
    })

    # Update cell colors
    for y in range(new_height):
        for x in range(new_width):
            color = img_array[y, x]
            requests.append({
                'updateCells': {
                    'rows': [{
                        'values': [{
                            'userEnteredFormat': {
                                'backgroundColor': {
                                    'red': color[0] / 255,
                                    'green': color[1] / 255,
                                    'blue': color[2] / 255
                                }
                            }
                        }]
                    }],
                    'fields': 'userEnteredFormat.backgroundColor',
                    'range': {
                        'sheetId': sheet.id,
                        'startRowIndex': y,
                        'endRowIndex': y + 1,
                        'startColumnIndex': x,
                        'endColumnIndex': x + 1
                    }
                }
            })

    # Adjust column widths to make cells square
    requests.append({
        'updateDimensionProperties': {
            'range': {
                'sheetId': sheet.id,
                'dimension': 'COLUMNS',
                'startIndex': 0,
                'endIndex': new_width
            },
            'properties': {
                'pixelSize': 3
            },
            'fields': 'pixelSize'
        }
    })

    # Execute batch update
    body = {'requests': requests}
    sheet.spreadsheet.batch_update(body)

# Usage
image_path = 'image.jpg'
sheet_name = 'pixel_art'
credentials_path = 'google.json'
new_width = 800 # Adjust these values based on your preference
new_height = 85  # Maintain aspect ratio: 56 ≈ (549/976) * 100

update_sheet_with_pixel_art(image_path, sheet_name, credentials_path, new_width, new_height)